In [30]:
import pandas as pd
import glob
import json
import os

In [47]:
meta_files = os.listdir("figure_metas/thesis")
file_id = 0

In [48]:
meta_files

['dqn_decay_lr_train.json',
 'tsp_comparison.json',
 'dqn_fixed_lr.json',
 'dqn_fixed_lr_loss.json',
 'dqn_lrdecay_graph.json',
 'tsp_comparison_w_random.json',
 'dqn_fixed_lr_logloss.json',
 'sac_fixed_lr.json',
 'kool_fixed_lr.json',
 'op_final_trainings.json',
 'sac_fixed_lr_alpha.json',
 'op_comparison_w_random.json',
 'dqn_decay_lr.json',
 'dqn_nstep.json',
 'sac_decay_lr.json',
 'sac_fixed_alpha.json',
 'kool_decay_lr.json',
 'sac_fixed_lr_actorloss.json',
 'sac_fixed_lr_criticlosslog.json',
 'pg_neg_objective.json',
 'pg_fixed_lr_loss.json',
 'ppo_batch_size.json',
 'kool_baselines.json',
 'op_comparison.json',
 'pg_fixed_lr.json',
 'sac_fixed_lr_alphaloss.json',
 'ppo_fixed_lr.json',
 'op_final_trainings_zoom.json',
 'sac_fixed_lr_fail.json',
 'a2c_critic_encoding.json',
 'tsp_final_trainings.json',
 'a2c_decay_lr.json',
 'tsp_final_trainings_zoom.json',
 'pg_decay_lr.json',
 'a2c_v3v1v1visited.json',
 'a2c_negations.json',
 'sac_fixed_lr_criticloss.json',
 'pg_batch_norm.json'

In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

# naming a layout theme for future reference
pio.templates["google"] = go.layout.Template(
    layout_colorway=['#FD3216','#00FE35','#00B5F6','#EEA6FB','#0DF9FF','#FF9616','#1CBE4F','#EEA6FB'] #Light24 #['#4285F4', '#DB4437', '#F4B400', '#0F9D58', '#185ABC', '#B31412', '#EA8600', '#137333', '#d2e3fc', '#ceead6']
) # 1CBE4F

# setting Google color palette as default
pio.templates.default = "google"

In [210]:
#### 
# get data file names

#figure_meta = {
#    'key_path': 'log_dir_thesis/csvs/rew', # updateloss, testreward, LR, avg_cost, updatelossactor, evalrew
#    #'key_paths': ["log_dir_thesis/csvs/testreward", "log_dir_thesis/csvs/testreward", "log_dir_thesis/csvs/testreward", "log_dir_thesis/csvs/testreward", "log_dir_thesis/csvs/testreward", "log_dir_thesis/csvs/avg_cost", "log_dir_thesis/csvs/avg_cost"],
#    'y_df_label': 'rew', # test/reward, LR, avg_cost, update/loss/critic2
#    #'y_df_labels': ["test/reward", "test/reward", "test/reward", "test/reward", "test/reward", "avg_cost", "avg_cost"],
#    'invert_y': [True, True, True, True, True, False, False],
#    'x_df_label': 'Steps',
#    #'x_multiplier': 2560,
#    'y_display_label': 'average solution length',
#    'x_display_label': 'number of samples trained', # number of samples trained, problem size
#    'run_names': ["run_129__20220501T214922_20220502T162901_eval", "run_126__20220429T203838_20220502T162059_eval", "run_167__20220501T094242_20220502T161457_eval", "run_127__20220501T213709_20220502T161756_eval", "run_128__20220429T203856_20220502T162316_eval", "run_k37__20220504T232308_eval", "run_k35__20220502T232038_eval"],
#    'line_names': ["DQN", "PG", "SAC", "A2C", "PPO", "PG Kool no baseline", "PG Kool with baseline"],
#    'plot_filename': 'tsp_comparisonsdfsdf',
#    'legend_title': 'Algorithm',
#    'is_reward_plot': False,
#    'y_type': 'linear',
#    'legend_x': 0.99, # 0.99
#    'legend_y': 0.55,  # 0.84
#    'min_x': 0,
#    'max_x': 100,
#    'min_y': 1.0, #  3.0
#    'max_y': 12.5,# 10.8
#    'tsp_lines': False,
#    'op_lines': False,
#    'kool_thinning': 1,
#    'marker_size': 10
#}

#meta_name = meta_files[file_id]
#file_id +=1
#with open("figure_metas/thesis/" + meta_name) as json_file: #_loss
#    figure_meta = json.load(json_file)

    
#with open('figure_metas/thesis_smaller/ppo_batch_size.json') as json_file: #_loss
#    figure_meta = json.load(json_file)

#figure_meta['y_display_label'] = 'alpha'

#figure_meta['x_display_label'] = 'number of samples trained'
#figure_meta['y_display_label'] = 'alpha learning rate' # 'average solution length' # 'learning rate' # 'loss'
#figure_meta['y_type'] = 'linear'
#figure_meta['legend_x'] = 0.99      # 0.99
#figure_meta['legend_y'] = 0.60      # 0.84
#figure_meta['is_reward_plot'] = False
#figure_meta['min_x'] = 0
#figure_meta['max_x'] = 12800000
#figure_meta['min_y'] = 3.7          #  3.0
#figure_meta['max_y'] = 4.6       # 10.8
#figure_meta['tsp_lines'] = False
#figure_meta['op_lines'] = False
#figure_meta['kool_thinning'] = 40
#figure_meta['marker_size'] = 1
#figure_meta['x_multiplier'] = 2560
#figure_meta['invert_y'] = False
#figure_meta['invert_y'] = [True, True, True, True, True, False, False]
#figure_meta['scientific'] = False
#figure_meta['is_kool_plot'] = True
#figure_meta['is_neg_loss_plot'] = True



if 'scientific' not in figure_meta.keys():
    figure_meta['scientific'] = False
    
if 'x_multiplier' not in figure_meta.keys():
    figure_meta['x_multiplier'] = 1

if 'is_kool_plot' not in figure_meta.keys():
    figure_meta['is_kool_plot'] = False
    
if 'is_neg_loss_plot' not in figure_meta.keys():
    figure_meta['is_neg_loss_plot'] = False



#print(figure_meta['key_paths'])
#print(figure_meta['y_df_labels'])
#print(figure_meta['invert_y'])
#print(figure_meta['run_names'])
#print(figure_meta['line_names'])


#figure_meta['plot_filename'] = 'tsp_final_trainings_zoom'
#figure_meta['run_names'] = ['run_129__20220501T214922', 'run_126__20220429T203838', 'run_167__20220501T094242', 'run_127__20220501T213709', 'run_128__20220429T203856', 'run_k37__20220503T151519', 'run_k35__20220502T172652']


#figure_meta['line_names'] = ['DQN', 'PG', 'SAC', 'A2C', 'PPO', 'PG Kool no baseline', 'PG Kool with baseline']
#figure_meta['invert_y'] = [True, True, True, True, True, False, False]

#figure_meta['key_paths'] = ['log_dir_thesis/csvs/testreward', 'log_dir_thesis/csvs/testreward', 'log_dir_thesis/csvs/testreward', 'log_dir_thesis/csvs/testreward', 'log_dir_thesis/csvs/testreward', 'log_dir_thesis/csvs/avg_cost', 'log_dir_thesis/csvs/avg_cost']
#figure_meta['y_df_labels'] = ['test/reward', 'test/reward', 'test/reward', 'test/reward', 'test/reward', 'avg_cost', 'avg_cost']
#figure_meta['is_reward_plot'] = True

#figure_meta['run_names'] = ['run_000__20220408T230926_20220428T010610', 'run_001__20220409T002803_20220428T022531', 'run_002__20220409T014651_20220428T034418', 'run_003__20220409T030406_20220428T050142']

#del figure_meta['x_multiplier']

#figure_meta['is_reward_plot'] = False
#figure_meta['invert_y'] = True
#figure_meta['y_type'] = 'log'
#figure_meta['y_display_label'] = 'loss'

#figure_meta['key_path'] = 'log_dir_thesis/csvs/testreward'
#figure_meta['y_df_label'] = 'test/reward'
#figure_meta['plot_filename'] = 'pg_fixed_lr'

#figure_meta['y_df_label'] = 'update/loss'
#figure_meta['x_multiplier'] = 2560 # 64 envs * 20 steps * 2 episodes per collect
#figure_meta['plot_filename'] = 'pg_fixed_lr_loss' # pg_fixed_lr_loss


filenames = figure_meta['run_names']
x_df_label = figure_meta['x_df_label']
y_display_label = figure_meta['y_display_label']
x_display_label = figure_meta['x_display_label']
plot_names = figure_meta['line_names']
x_mult = figure_meta['x_multiplier']
is_reward_plot = figure_meta['is_reward_plot']
invert_y = figure_meta['invert_y']
y_type = figure_meta['y_type']
legend_x = figure_meta['legend_x']
legend_y = figure_meta['legend_y']
min_x = figure_meta['min_x']
max_x = figure_meta['max_x']
min_y = figure_meta['min_y']
max_y = figure_meta['max_y']
tsp_lines = figure_meta['tsp_lines']
op_lines = figure_meta['op_lines']
kool_thinning = figure_meta['kool_thinning']
marker_size = figure_meta['marker_size']
scientific = figure_meta['scientific']

plot_dfs = []
filepaths = []
if 'key_paths' in figure_meta.keys() and len(figure_meta['key_paths']) > 0:
    assert(len(figure_meta['key_paths']) == len(filenames))
    y_df_labels = figure_meta['y_df_labels']
    for path_str, filename in zip(figure_meta['key_paths'], filenames):
        filepaths.append(path_str + "/" + filename + ".csv")
else:
    path = figure_meta['key_path']
    filepaths = [path + "/" + filename + ".csv" for filename in filenames]
    if not isinstance(invert_y, list):
        invert_y = [invert_y] * len(filepaths)
    y_df_labels = [figure_meta['y_df_label']] * len(filepaths)
    
for i, filepath in enumerate(filepaths):
    df = pd.read_csv(filepath, sep=';')
    if invert_y[i]:
        df[y_df_labels[i]] = -df[y_df_labels[i]]
    plot_dfs.append(df)

In [211]:
# https://plotly.com/python/plot-data-from-csv/
# https://stackoverflow.com/questions/64500472/plot-multiple-pandas-dataframes-in-one-graph-python-plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.kaleido.scope.mathjax = None # https://github.com/plotly/plotly.py/issues/3469

fig = go.Figure()
for i, (name, df) in enumerate(zip(plot_names, plot_dfs)):
    if "Kool" in name or figure_meta['is_kool_plot']:
        fig = fig.add_trace(go.Scatter(x = df[x_df_label][::kool_thinning] * x_mult, # 40
                                   y = df[y_df_labels[i]][::kool_thinning], 
                                   name = name,
                                   marker=dict(size=marker_size),
                                   line=dict(width=3))) # , marker=dict(size=10)
    else:
        fig = fig.add_trace(go.Scatter(x = df[x_df_label][::1] * x_mult,
                                   y = df[y_df_labels[i]][::1], 
                                   name = name,
                                   marker=dict(size=marker_size),
                                   line=dict(width=3)))

fig.update_layout(
    font_family="libertine",
    font_size=28,
    autosize=False,
    width=2000,
    height=666,
    margin=dict(
        l=120,
        r=70,
        b=80,
        t=30,
        pad=4
    ),
    yaxis=dict(
        title_text=y_display_label,
        type="linear" if is_reward_plot else y_type,
        range=(min_y, max_y), 
        rangemode="tozero",
        tickformat='f' if y_type == 'log' else ('.0e' if scientific else None), #'.0e'
        
        ticktext=["-0.1", "-1", "-10", "-100", "-1000", "-10k", "-100k", "-1M", "-10M"] if figure_meta['is_neg_loss_plot'] else None,
        tickvals=[0.1, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000] if figure_meta['is_neg_loss_plot'] else None,
        #tickmode="array",
        #titlefont=dict(size=30),
    ),
    xaxis=dict(
        title_text=x_display_label,
        range=None if is_reward_plot else (min_x, max_x), # x_mult*df[x_df_label].iloc[-1]
        rangemode="tozero",
        #ticktext=["Very long label", "long label", "3", "label"],
        #tickvals=[1, 2, 3, 4],
        #tickmode="array",
        #titlefont=dict(size=30),
    ),
    title=dict(
        #text=title,
    ),
    legend=dict(
        yanchor="top",
        y=legend_y, # 84   6
        xanchor="right",
        x=legend_x, #0.99,
        bgcolor='rgba(255,255,255,0.8)'
    ),
    legend_title_text=figure_meta['legend_title'],
    #paper_bgcolor="LightSteelBlue",
)
# https://plotly.com/python/horizontal-vertical-shapes/
if tsp_lines:
    fig.add_hline(y=10.43, line_dash="dot", annotation_text="random", annotation_position="bottom right")
    fig.add_hline(y=3.84, line_dash="dot", annotation_text="optimal", annotation_position="bottom right")
    
if op_lines:
    fig.add_hline(y=1.58, line_dash="dot", annotation_text="random", annotation_position="bottom right")
    fig.add_hline(y=5.39, line_dash="dot", annotation_text="optimal", annotation_position="bottom right")

#fig.add_hline(y=3.84, line_dash="dot", annotation_text="optimal (20)", annotation_position="bottom right")
#fig.add_hline(y=5.70, line_dash="dot", annotation_text="optimal (50)", annotation_position="bottom right")
#fig.add_hline(y=7.76, line_dash="dot", annotation_text="optimal (100)", annotation_position="bottom right")

fig.show()
fig.write_image("figures/" + figure_meta['plot_filename'] + ".pdf")

with open("figure_metas/" + figure_meta['plot_filename'] + ".json", "w") as fp:
    json.dump(figure_meta,fp)
    